![Renode](https://antmicro.com/OpenSource/assets/images/projects/renode.png)

## Requirements to run locally (if you really want!)

* pyrenode (super simple script a variant of renode-pytest)
* Jupyter notebook installed and running locally:

```
python3 -m jupyter notebook \
  --NotebookApp.allow_origin='https://colab.research.google.com' \
  --port=8888 \
  --NotebookApp.port_retries=0
```

* following this guide: https://research.google.com/colaboratory/local-runtimes.html

In [ ]:
import ipywidgets as widgets
import os

In [ ]:
!rm renode.tar.gz
!wget https://www.dropbox.com/s/b4eqhiqxgnn1vcx/renode.tar.gz
!mkdir -p renode && cd renode && tar -xzf ../renode.tar.gz --strip 1
os.environ['PATH'] = os.getcwd()+"/renode:"+os.environ['PATH']

In [ ]:
!pip3 install git+https://github.com/mgielda/pyrenode robotframework==3.1 netifaces requests psutil pyyaml matplotlib pandas numpy

## Housekeeping

Some initialization. Import a simplistic pyrenode lib to talk to Renode over telnet, then shutdown Renode just in case it was running and create a new instance.

In [ ]:
from pyrenode import *
shutdown_renode()
connect_renode() # this sets up a log file, and clears the simulation (just in case)

In [ ]:
tell_renode('i @scripts/single-node/hifive_unleashed.resc')
print(read_until('(hifive-unleashed)').strip())
tell_renode('machine EnableProfiler "demo"')
print(read_until('(hifive-unleashed)').strip())
tell_renode('s')
print(read_until('Starting emulation...').strip())

In [ ]:
shutdown_renode()
expect_log("Disposed.") # to be sure if buffer to dump file is flushed
!python3 renode/tools/metrics-analyzer/metrics-analyzer.py renode/demo

In [ ]:
from IPython.display import Image
Image('instructions.png')

In [ ]:
from IPython.display import Image
Image('memory.png')

In [ ]:
from IPython.display import Image
Image('exceptions.png')

In [ ]:
from IPython.display import Image
Image('peripherals.png')